In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import modules.eda as Detective
import pandas as pd
import warnings
import numpy as np
import emojis
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier

warnings.filterwarnings('ignore')

In [3]:
def expandEmojisDecode(pcomment: str):
    expand_emojis = ''
    for e in emojis.get(pcomment):
        amount = pcomment.count(e)
        expand_emojis += (f"{emojis.decode(e)[1:-1]} ")*amount
        
    return expand_emojis.strip()

In [4]:
reviews = pd.read_csv("./data/normalize_reviews.csv").fillna("")
reviews = reviews[['raw_comment', 'normalize_comment', 'emoji', 'label']]

reviews.head()

,raw_comment,normalize_comment,emoji,label
0,Giao hàng kh đúng cần phê bình hjjjjjhhd...,giao hàng không đúng cần phê bình,,0
1,Chất lượng sản phẩm tạm được. Giao...,chất lượng sản phẩm tạm được giao ...,,0
2,Ko có lắc tay như hình,không có lắc tay như hình,,0
3,Giao hàng lâu. Bảo có lắc tay mà k thâ...,giao hàng lâu bảo có lắc tay mà không ...,,0
4,"Mình mua 2 cái, một dùng ok. Một cái k...",mua cái một dùng ok một cái không chạ...,😢,0


Hiện tại, dataset chúng ta có hai feature là `normalize_comment` và `emoji`, chúng ta có thể xây dựng một sentiment analysis classifier bằng hai cách:
  * **Cách 1**: Chúng ta có thể đem feature `emoji` để thêm vào phần đầu hoặc cuối của feature `normalize_comment`. Sau đó tiến hành đào tạo một model duy nhất với input là feature `normalize_comment` đã được thêm vào feature `emoji`.
    * Ưu điểm: Quá trình đào tạo model đỡ tốn thời gian hơn do ta chỉ thực hiện build một model duy nhất.
    * Nhược điểm:
      * Có khả năng làm suy giảm sức mạnh của model, nhiều người họ còn xây dưng riêng một **emoji classifier** bởi vì theo họ nhiều khi emoji đó mang lại ý nghĩa thậm chí còn cao hơn so với ngữ nghĩa của comment đó. Trong khi việc đào tạo một emoji classifier đỡ vất vã hơn.
      * Ta phải tốn thời gian vào việc xây dựng thêm một model riêng sau đó lại tổng hợp kết quả của emoji classifier model này với sentiment classifier model ban đầu.
  * **Cách 2**: Ta có thể xây dựng hai model riêng biệt, một sentiment analysis model đảm nhận nhiệm vụ phân lớp cho câu chữ và một emoji analysis model đảm nhận nhiệm vụ phân lớp trên emoji. Sau đó ta kết hợp hai model này để cho ra kết quả phân lớp cuối cùng.
    * Ưu điểm: Kết quả phân lớp cho ra có khả năng chính xác hơn, ta có thể linh hoạt trong việc thiếp lập trọng số để ưu tiên giá trị của model nào mang lại lợi ích cao hơn cho kết quả ở bước kết hợp hai model lại để dự đoán.
    * Nhược điểm:
      * Vất vả hơn về mặt tiền xử lí dữ liệu, tốn thêm thời gian phân tích và thiết kế thêm một emoji classifier.
      * Phần lớn trong dataset hiện tại, số comment chứa emoji không nhiều. Như project 2 ta đã thấy toàn bộ dataset của ta chỉ khoảng 1300 comment là có chứa emoji kèm theo, ta có thể khắc phục bằng một trong những cách:
        * Crawl thêm data, nhưng việc này không khả khi. Ta đã đề cập rằng số comment chứa emoji rất ít trong một dataset nên việc ta hi sinh thời gian chỉ để trích xuất một phần nhỏ trong một dataset cực lớn sau khi crawl về có thể phí phạm.
        * Sử dụng kĩ thuật **Data Upsampler** với package của IBM `from imblearn.over_sampling import RandomOverSampler`.

    $\Rightarrow$ Hướng giải quyết: Quả thật ta có khoảng 1300 observer thì là một con số không quá ít cũng không quá nhiều, số lượng emoji chứa trong một comment cũng không nhiều (không tính các duplicate emoji). Bài toán của chúng ta đơn thuần chỉ là phân tích các emoji ra hai class là negative và positive. Hãy cùng nhìn lại một dataset nổi tiếng khác là **Iris**, ta có 3 class ở target variable và 150 observe. Vậy ta vẫn có thể build một classifier model đủ tốt nếu input của ta cũng **đủ tốt**.

# Emoji sentiment analysis

Như vậy chiến lược đào tạo một Emoji sentiment analysis của chúng ta sẽ như sau:
  * Bước 1: Tiến hành lấy toàn bộ các emoji trong các comment và lưu trong cấu trúc dữ liệu `set`, mục tiêu ta cần biết có bao nhiêu unique emoji trong dataset của chúng ta.
  * Bước 2: Tạo 2 dictionary, một dictionary có key là emoji và value là index của emoji đó trong `set`, dictionary thứ hai có key là index của emoji trong `set` và value là emoji.
  * Bước 3: Giả sử có $n$ unique emoji trong dataset, ta sẽ chỉ quan tâm đến các comment chứa emoji, và với từng comment như vậy ta tạo một vector $n$ phần tử sau đó áp dụng phương pháp **Bag of words** lên vector này.
  * Bước 4: Xem xét $n$ có lớn không, nếu quá lớn thì có thể áp dụng các phương pháp giảm chiều dữ liệu.
  * Bước 5: Dùng các vector $n$ phần tử + label của chúng sau khi đã qua bước 4 để đào tạo một classifier model.

Bây giờ ta sẽ tiến hành lọc ra nhựng comment mà có chứa emoji dựa vào feature `emoji`. Sau đó lưu các observe này vào biến `emojis`.

In [5]:
data = reviews[reviews['emoji'] != ""]

data

,raw_comment,normalize_comment,emoji,label
4,"Mình mua 2 cái, một dùng ok. Một cái k...",mua cái một dùng ok một cái không chạ...,😢,0
15,Giao sai màu sai size có 1 dép lông size 3...,giao sai màu sai size có dép lông size à ...,🤬,0
30,Đơn hàng đã thanh toán airpay rồi mà sh...,đơn hàng thanh toán mà giao cho người l...,🙄,0
43,Áo croptop freesize rộng với mình \nMìn...,áo rộng mình kg,👍,0
47,Khá buồn . Đặt 2 cái đều không chạy ...,khá buồn đặt cái đều không chạy giơ...,♀️ 🤷,0
...,...,...,...,...
14184,"Không mua hối hận đừng kêu nhá haha, ...",không mua hối hận đừng kêu nhá hàng ...,😂,1
14186,khuyên xinh lắm lúc mình đặt còn đươ...,khuyên xinh lắm lúc đặt còn được dea...,😘,1
14193,đẹp rẻ chất lượng vô cùng xịn 😗😗😗 ma...,đẹp rẻ chất lượng vô cùng xịn mãi u...,😗,1
14194,"Đẹp, sẽ ủng hộ tiếp nhé 👕👕👕👕👕👕👕👕👕👕👕👕👕👕...",đẹp sẽ ủng hộ tiếp nhé,👕,1


Package **emojis** chỉ hỗ trợ chức năng tách emoji ra khỏi text, nó không hỗ trợ thống kê là một emoji xuất hiện bao nhiêu lần trong text đó. Bây giờ ta sẽ tách các emoji trong comment ra nhưng vẫn bảo toàn về mặt số lượng ban đầu của nó trong feature `raw_comment` và lưu vào feature `emoji`.

In [6]:
data['decode_emoji'] = data['raw_comment'].apply(lambda cmt: expandEmojisDecode(cmt))

data

,raw_comment,normalize_comment,emoji,label,decode_emoji
4,"Mình mua 2 cái, một dùng ok. Một cái k...",mua cái một dùng ok một cái không chạ...,😢,0,cry
15,Giao sai màu sai size có 1 dép lông size 3...,giao sai màu sai size có dép lông size à ...,🤬,0,cursing_face cursing_face cursing_face
30,Đơn hàng đã thanh toán airpay rồi mà sh...,đơn hàng thanh toán mà giao cho người l...,🙄,0,roll_eyes
43,Áo croptop freesize rộng với mình \nMìn...,áo rộng mình kg,👍,0,thumbsup thumbsup thumbsup thumbsup thumbsup t...
47,Khá buồn . Đặt 2 cái đều không chạy ...,khá buồn đặt cái đều không chạy giơ...,♀️ 🤷,0,female_sign shrug
...,...,...,...,...,...
14184,"Không mua hối hận đừng kêu nhá haha, ...",không mua hối hận đừng kêu nhá hàng ...,😂,1,joy
14186,khuyên xinh lắm lúc mình đặt còn đươ...,khuyên xinh lắm lúc đặt còn được dea...,😘,1,kissing_heart kissing_heart kissing_heart
14193,đẹp rẻ chất lượng vô cùng xịn 😗😗😗 ma...,đẹp rẻ chất lượng vô cùng xịn mãi u...,😗,1,kissing kissing kissing kissing kissing kissin...
14194,"Đẹp, sẽ ủng hộ tiếp nhé 👕👕👕👕👕👕👕👕👕👕👕👕👕👕...",đẹp sẽ ủng hộ tiếp nhé,👕,1,tshirt tshirt tshirt tshirt tshirt tshirt tshi...


Bước này, ta thực hiện chia `emojis` ra thành hai tập training data và test data với test data chiếm 20% số lượng các observer của `emojis`.

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data['decode_emoji'], data['label'], test_size=0.2, random_state=42)

In [8]:
X_train

1521                                            heart_eyes
11485    hearts hearts hearts kissing_heart kissing_hea...
3467                                              relieved
7740     persevere persevere persevere persevere persev...
12870                                          heart heart
                               ...                        
12351                                           gift_heart
12799                                                heart
13056                                            wink hugs
10730    blush blush blush blush blush blush blush blus...
13032                                                 wink
Name: decode_emoji, Length: 1026, dtype: object

Bây giờ, chúng ta thực hiện kĩ thuật **vectorizing text**, ta áp dụng lần lượt hai phương pháp là **Bag of Words** và **TF-IDF**.

In [9]:
# Bag of words
bow_vec = CountVectorizer()
bow_emojis = bow_vec.fit_transform(X_train)

print(bow_vec.get_feature_names())
print(bow_emojis.toarray())

['100', 'anger', 'angry', 'ballet_shoes', 'bangbang', 'birthday', 'blossom', 'blue_heart', 'blush', 'bouquet', 'bow', 'broken_heart', 'butterfly', 'cake', 'camera', 'carrot', 'cherry_blossom', 'clap', 'cloud_with_rain', 'clown_face', 'cold_face', 'cold_sweat', 'compass', 'confounded', 'confused', 'cow', 'crossed_fingers', 'cry', 'crying_cat_face', 'cursing_face', 'disappointed', 'disappointed_relieved', 'dizzy', 'dizzy_face', 'dog', 'dog2', 'dollar', 'eagle', 'expressionless', 'face_with_thermometer', 'facepalm', 'fallen_leaf', 'fearful', 'female_sign', 'fire', 'fist_left', 'floppy_disk', 'flushed', 'four_leaf_clover', 'free', 'frowning_face', 'fu', 'ghost', 'gift_heart', 'golf', 'green_heart', 'green_salad', 'grimacing', 'grin', 'grinning', 'haircut', 'hand_over_mouth', 'hatched_chick', 'heart', 'heart_eyes', 'heart_eyes_cat', 'heartbeat', 'heartpulse', 'hearts', 'heavy_check_mark', 'heavy_heart_exclamation', 'hibiscus', 'high_brightness', 'hugs', 'information_desk_person', 'innocent'

In [10]:
tfidf_vec = TfidfVectorizer()
tfidf_emojis = tfidf_vec.fit_transform(X_train)

print(tfidf_emojis.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
